# Multi Environment Config Management - SSM Backend

This is a follow up of [Multi Environment Config Management](https://github.com/MacHu-GWU/config_patterns-project/blob/main/example/multi_env_json/multi_environment_config.ipynb). In this article, we will introduce using [AWS System Manager Parameter Store](https://docs.aws.amazon.com/systems-manager/latest/userguide/systems-manager-parameter-store.html) as the backend to manage your multi-environment configurations.

We have prepared three versions of the config (v1, v2, v3) for testing purposes. Now, let's take a preview of the config data for each version.

In [1]:
import json
from rich import print as rprint

def jprint(data: dict):
    rprint(json.dumps(data, indent=4))

In [2]:
from python_lib.config_paths import (
    path_config_v1,
    path_config_secret_v1,
    path_config_v2,
    path_config_secret_v2,
    path_config_v3,
    path_config_secret_v3,
)

print("------ Version 1 ------")
rprint(path_config_v1.read_text())
rprint(path_config_secret_v1.read_text())

print("------ Version 2: ------")
rprint(path_config_v2.read_text())
rprint(path_config_secret_v2.read_text())

print("------ Version 3: ------")
rprint(path_config_v3.read_text())
rprint(path_config_secret_v3.read_text())

------ Version 1 ------


# content of config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
        "*.project_name": "my_project"
    },
    "dev": {
        "username": "dev.user.v1"
    },
    "prod": {
        "username": "prod.user.v1"
    }
}

# content of secret_config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
    },
    "dev": {
        "password": "dev.password"
    },
    "prod": {
        "password": "prod.password"
    }
}

------ Version 2: ------


# content of config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
        "*.project_name": "my_project"
    },
    "dev": {
        "username": "dev.user.v2"
    },
    "prod": {
        "username": "prod.user.v2"
    }
}

# content of secret_config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
    },
    "dev": {
        "password": "dev.password"
    },
    "prod": {
        "password": "prod.password"
    }
}

------ Version 3: ------


# content of config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
        "*.project_name": "my_project"
    },
    "dev": {
        "username": "dev.user.v3"
    },
    "prod": {
        "username": "prod.user.v3"
    }
}

# content of secret_config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
    },
    "dev": {
        "password": "dev.password"
    },
    "prod": {
        "password": "prod.password"
    }
}

Similar to what we have done in [Multi Environment Config Management](https://github.com/MacHu-GWU/config_patterns-project/blob/main/example/multi_env_json/multi_environment_config.ipynb), we declared the config data model as below.

In [3]:
import os
# content of config_define.py
# -*- coding: utf-8 -*-

import typing as T
import os
import dataclasses

from config_patterns.patterns.multi_env_json.api import (
    BaseEnvEnum, # the base class of the environment name enum class
    BaseEnv, # the base class of the per environment object
    BaseConfig, # the base class of the all-in-one config object
)


class EnvEnum(BaseEnvEnum):
    dev = "dev" # development
    prod = "prod" # production


@dataclasses.dataclass
class Env(BaseEnv):
    username: T.Optional[str] = dataclasses.field(default=None)
    password: T.Optional[str] = dataclasses.field(default=None)

    @classmethod
    def from_dict(cls, data: dict):
        """
        This method defines how to create an instance of this class from a dict.

        Example:

            >>> Env.from_dict({"username": "user1", "password": "pass1"})
        """
        return cls(**data)

    @property
    def login_info(self) -> str:
        """
        This is a sample derived attribute.
        """
        return f"Hello {self.username}, please enter your password: "


@dataclasses.dataclass
class Config(BaseConfig):
    @property
    def dev(self) -> Env:
        """
        A shortcut to get the dev environment config object.
        """
        return self.get_env(EnvEnum.dev)

    @property
    def prod(self) -> Env:
        """
        A shortcut to get the dev environment config object.
        """
        return self.get_env(EnvEnum.prod)

    @classmethod
    def get_current_env(cls) -> str:
        """
        You may want a smarter way to determine the current environment.
        For example, you may define the local laptop is ``dev``, and the
        virtual machine is ``prod``.
        """
        if "IS_VM" in os.environ:
            return EnvEnum.prod.value
        else:
            return EnvEnum.dev.value

    @property
    def env(self) -> Env:
        """
        This is a shortcut to get the current environment object.
        """
        return self.get_env(self.get_current_env())

In this tutorial, we utilize [moto](https://docs.getmoto.org/en/latest/docs/getting_started.html) to mock AWS services. Therefore, you don't need to set up a real AWS account and can focus on the concepts.

In [4]:
import moto
from boto_session_manager import BotoSesManager
from s3pathlib import S3Path

# mock related AWS services
mock_ssm = moto.mock_ssm()
mock_sts = moto.mock_sts()
mock_ssm.start()
mock_sts.start()

# create a boto session manager object 
bsm = BotoSesManager(region_name="us-east-1")

## Read the Config Object from Local JSON File

First, we read the config version 1 from local JSON file.

In [5]:
config_v1 = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    path_config=path_config_v1,
    path_secret_config=path_config_secret_v1,
)
rprint(config_v1)

Config(
    data={
        '_shared': {'*.project_name': 'my_project'},
        'dev': {'username': 'dev.user.v1'},
        'prod': {'username': 'prod.user.v1'}
    },
    secret_data={'_shared': {}, 'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    Env=<class '__main__.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    version='local',
    _applied_data={
        'dev': {'username': 'dev.user.v1', 'project_name': 'my_project'},
        'prod': {'username': 'prod.user.v1', 'project_name': 'my_project'}
    },
    _applied_secret_data={'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    _merged={
        'dev': {'username': 'dev.user.v1', 'project_name': 'my_project', 'password': 'dev.password'},
        'prod': {'username': 'prod.user.v1', 'project_name': 'my_project', 'password': 'prod.password'}
    }
)

The config object has two built-in attributes ``project_name`` and ``env_name``.

In [6]:
print(f"project_name: {config_v1.project_name}")
print(f"dev env_name: {config_v1.dev.env_name}")
print(f"prod env_name: {config_v1.prod.env_name}")

project_name: my_project
dev env_name: dev
prod env_name: prod


The config object also has a built-in derived attribute ``parameter_name``. It is the normalized name (convert to lower case and snake case (underscore only)) for config deployment resources name of your backend.

In [7]:
print(f"parameter_name: {config_v1.parameter_name}")

parameter_name: my_project


## Deploy Config to SSM Parameter Store

Now, you can use the ``Config.deploy()`` method to deploy the config object to SSM Parameter Store. The ``parameter_with_encryption=True|False`` argument has to be specified to use SSM backend. It creates an all-in-one deployment containing all environment data as a backup for disaster recovery or compliance requirements. Additionally, it creates individual per-environment deployments, ensuring that each environment can only access its specific configuration data and cannot access the configuration of other environments.

In [8]:
deployment_list = config_v1.deploy(
    bsm=bsm, 
    parameter_with_encryption=True,
    # these two arguments are optional
    tags={"project_name": config_v1.project_name},
    verbose=True,
)

+----- ⏱ 🟢 Start 'deploy config to SSM parameter' -----------------------------+
| 
| 🚀️ deploy SSM Parameter 'my_project' ...
| preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project/description?region=us-east-1&tab=Table
| successfully deployed version 1
| 
+----- ⏰ 🟢 End 'deploy config to SSM parameter', elapsed = 0.18 sec -----------+
+----- ⏱ 🟢 Start 'deploy config to SSM parameter' -----------------------------+
| 
| 🚀️ deploy SSM Parameter 'my_project-dev' ...
| preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-dev/description?region=us-east-1&tab=Table
| successfully deployed version 1
| 
+----- ⏰ 🟢 End 'deploy config to SSM parameter', elapsed = 0.00 sec -----------+
+----- ⏱ 🟢 Start 'deploy config to SSM parameter' -----------------------------+
| 
| 🚀️ deploy SSM Parameter 'my_project-prod' ...
| preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-prod/descript

If you are trying to deploy the same config data with no change, ``config_patterns`` library can automatically detect that and skip the deployment.

In [9]:
deployment_list = config_v1.deploy(
    bsm=bsm, 
    parameter_with_encryption=True,
    # these two arguments are optional
    tags={"project_name": config_v1.project_name},
    verbose=True,
)

+----- ⏱ 🟢 Start 'deploy config to SSM parameter' -----------------------------+
| 
| 🚀️ deploy SSM Parameter 'my_project' ...
| preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project/description?region=us-east-1&tab=Table
| parameter data is the same as existing one, do nothing.
| 
+----- ⏰ 🟢 End 'deploy config to SSM parameter', elapsed = 0.01 sec -----------+
+----- ⏱ 🟢 Start 'deploy config to SSM parameter' -----------------------------+
| 
| 🚀️ deploy SSM Parameter 'my_project-dev' ...
| preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-dev/description?region=us-east-1&tab=Table
| parameter data is the same as existing one, do nothing.
| 
+----- ⏰ 🟢 End 'deploy config to SSM parameter', elapsed = 0.00 sec -----------+
+----- ⏱ 🟢 Start 'deploy config to SSM parameter' -----------------------------+
| 
| 🚀️ deploy SSM Parameter 'my_project-prod' ...
| preview at: https://us-east-1.console.aws.amazon.com/sys

## Read Config from SSM Parameter Store

Now, you can use the Config.read() method to readthe config object from SSM Parameter store. The ``parameter_with_encryption=True|False`` argument has to be specified to use SSM backend. If you want to read the all-in-one config object, then you could use ``parameter_name="${parameter_name}"``. If you want to read the config object of specific environment, you could use ``parameter="${parameter_name}-${env_name}"``.

In the first example, we are the human developer reading the all-in-one config object. So it should have access to all environment.

In [10]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    bsm=bsm,
    parameter_name="my_project",
    parameter_with_encryption=True,
)
print("all in one config object:")
rprint(config)

all in one config object:


Config(
    data={
        '_shared': {'*.project_name': 'my_project'},
        'dev': {'username': 'dev.user.v1'},
        'prod': {'username': 'prod.user.v1'}
    },
    secret_data={'_shared': {}, 'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    Env=<class '__main__.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    version='1',
    _applied_data={
        'dev': {'username': 'dev.user.v1', 'project_name': 'my_project'},
        'prod': {'username': 'prod.user.v1', 'project_name': 'my_project'}
    },
    _applied_secret_data={'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    _merged={
        'dev': {'username': 'dev.user.v1', 'project_name': 'my_project', 'password': 'dev.password'},
        'prod': {'username': 'prod.user.v1', 'project_name': 'my_project', 'password': 'prod.password'}
    }
)

In this example, we are the machine application reading the ``dev`` environment config object. We should not be able to access ``prod`` environment config data.

In [11]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    bsm=bsm,
    parameter_name="my_project-dev",
    parameter_with_encryption=True,
)
print("dev config object:")
rprint(config)

dev config object:


Config(
    data={'_shared': {'*.project_name': 'my_project'}, 'dev': {'username': 'dev.user.v1'}},
    secret_data={'_shared': {}, 'dev': {'password': 'dev.password'}},
    Env=<class '__main__.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    version='1',
    _applied_data={'dev': {'username': 'dev.user.v1', 'project_name': 'my_project'}},
    _applied_secret_data={'dev': {'password': 'dev.password'}},
    _merged={'dev': {'username': 'dev.user.v1', 'project_name': 'my_project', 'password': 'dev.password'}}
)

In [16]:
config.prod

KeyError: 'prod'

## Deploy a New Version of Config

When you deploy a new version of the config, it creates a new version of the parameter. It is important to note that, [AWS SSM maintains up to 100 versions of a parameter](https://docs.aws.amazon.com/systems-manager/latest/userguide/sysman-paramstore-versions.html). After you have created 100 versions of a parameter, each time you create a new version, the oldest version of the parameter is removed from history to make room for the new version.

In [12]:
config_v2 = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    path_config=path_config_v2,
    path_secret_config=path_config_secret_v2,
)
rprint(config_v2)

deployment_list = config_v2.deploy(
    bsm=bsm, 
    parameter_with_encryption=True,
    # these two arguments are optional
    tags={"project_name": config_v2.project_name},
    verbose=True,
)

Config(
    data={
        '_shared': {'*.project_name': 'my_project'},
        'dev': {'username': 'dev.user.v2'},
        'prod': {'username': 'prod.user.v2'}
    },
    secret_data={'_shared': {}, 'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    Env=<class '__main__.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    version='local',
    _applied_data={
        'dev': {'username': 'dev.user.v2', 'project_name': 'my_project'},
        'prod': {'username': 'prod.user.v2', 'project_name': 'my_project'}
    },
    _applied_secret_data={'dev': {'password': 'dev.password'}, 'prod': {'password': 'prod.password'}},
    _merged={
        'dev': {'username': 'dev.user.v2', 'project_name': 'my_project', 'password': 'dev.password'},
        'prod': {'username': 'prod.user.v2', 'project_name': 'my_project', 'password': 'prod.password'}
    }
)

+----- ⏱ 🟢 Start 'deploy config to SSM parameter' -----------------------------+
| 
| 🚀️ deploy SSM Parameter 'my_project' ...
| preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project/description?region=us-east-1&tab=Table
| successfully deployed version 2
| 
+----- ⏰ 🟢 End 'deploy config to SSM parameter', elapsed = 0.00 sec -----------+
+----- ⏱ 🟢 Start 'deploy config to SSM parameter' -----------------------------+
| 
| 🚀️ deploy SSM Parameter 'my_project-dev' ...
| preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-dev/description?region=us-east-1&tab=Table
| successfully deployed version 2
| 
+----- ⏰ 🟢 End 'deploy config to SSM parameter', elapsed = 0.00 sec -----------+
+----- ⏱ 🟢 Start 'deploy config to SSM parameter' -----------------------------+
| 
| 🚀️ deploy SSM Parameter 'my_project-prod' ...
| preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-prod/descript

## Delete and Clean Up

Normally, it is not necessary to delete any config deployments in AWS SSM. [There's no cost using SSM Parameter store in standard tier](https://aws.amazon.com/systems-manager/pricing/). If you accidentally deploy malformed config data, there is no need to delete it. Instead, you can simply create a new deployment with the corrected configuration.

To clean up all config objects in all environments, including the historical versions, you can retrieve the all-in-one object and then use the ``config.delete()`` method. By default, when you delete the config, all historical versions is deleted. The ``include_history=True`` is only for S3 Backend, this arugment is ignored if using SSM backend.

In [15]:
deployment_list = config_v2.delete(
    bsm=bsm,
    use_parameter_store=True,
)

+----- ⏱ 🟢 Start 'delete config from SSM parameter' ---------------------------+
| 
| 🗑️ delete SSM Parameter 'my_project' ...
| verify at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project/description?region=us-east-1&tab=Table
| done!
| 
+----- ⏰ 🟢 End 'delete config from SSM parameter', elapsed = 0.01 sec ---------+
+----- ⏱ 🟢 Start 'delete config from SSM parameter' ---------------------------+
| 
| 🗑️ delete SSM Parameter 'my_project-dev' ...
| verify at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-dev/description?region=us-east-1&tab=Table
| done!
| 
+----- ⏰ 🟢 End 'delete config from SSM parameter', elapsed = 0.00 sec ---------+
+----- ⏱ 🟢 Start 'delete config from SSM parameter' ---------------------------+
| 
| 🗑️ delete SSM Parameter 'my_project-prod' ...
| verify at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-prod/description?region=us-east-1&tab=Table
| done!
| 
+----- ⏰ 🟢 En

## Summary

AWS SSM backend is perfect for config data store. It is my personal favorite backend. However, it has the limit to retain only 100 historical versions. If you want unlimited backup of all historical version, consider using S3 backend along with SSM backend. Please refer to this document:

- [Multi Environment Config Management - S3 Backend](https://github.com/MacHu-GWU/config_patterns-project/blob/main/example/multi_env_json/multi_environment_config_with_s3_backend.ipynb)
